In [1]:
#we import our necessary modules
import pandas as pd
import os

In [2]:
#we read in our csv's of raw data
birth_rate_path = os.path.join("Birth_Rates.csv")
food_production_path = os.path.join("Food_Production.csv")
currency_path = os.path.join("currency.csv")
currencies_path = os.path.join("currencies.csv")
birth_rates_raw = pd.read_csv(birth_rate_path)
food_production_raw = pd.read_csv(food_production_path)
currency = pd.read_csv(currency_path)
currencies = pd.read_csv(currencies_path)

In [3]:
#we clean up the currency dataframe
currency = currency.drop(['Unnamed: 0'], axis=1)
currency = currency.rename(columns={'0':'Date','1':'Rate','2':'Currency'})

currency = currency[currency.Date != "{{point.Date}}"]
currency = currency[currency.Date != "Average"]
currency = currency[currency.Rate != "{{point.Rate}}"]
currency = currency[currency.Rate != "{{results.average}}"]

currency = currency[['Currency', 'Date', 'Rate']]
for index, row in currency.iterrows():
    row['Date'] = row['Date'][-4:]
currency['Rate'] = pd.to_numeric(currency['Rate'])
currency_clean = currency.groupby(['Currency','Date']).mean().reset_index()
currency_clean.dropna()
currency_clean

,Currency,Date,Rate
0,AED UAE Dirham,1990,0.780672
1,AED UAE Dirham,1991,0.778778
2,AED UAE Dirham,1992,0.734887
3,AED UAE Dirham,1993,0.679575
4,AED UAE Dirham,1994,0.731567
...,...,...,...
1793,ZAR South African Rand,2016,0.743605
1794,ZAR South African Rand,2017,0.766849
1795,ZAR South African Rand,2018,0.747787
1796,ZAR South African Rand,2019,0.695524


In [4]:
#we clean up the birth rates dataframe
birth_rates_raw = birth_rates_raw[['Country Name', 'Country Code','1990','1991','1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']]
birth_rates = pd.DataFrame()
years = ['1990','1991','1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']
for index, row in birth_rates_raw.iterrows():
    for year in years:
        birth_rates = birth_rates.append([[row[0],row[1],year,row[year]]])
birth_rates = birth_rates.rename(columns={0:'Country', 1:'Country Code', 2:'Year', 3:'Birth Rate'})
birth_rates = birth_rates.reset_index(drop = True)
#we drop rows that we don't have data for
birth_rates = birth_rates.dropna()
birth_rates

,Country,Country Code,Year,Birth Rate
0,Aruba,ABW,1990,19.571
1,Aruba,ABW,1991,19.021
2,Aruba,ABW,1992,18.446
3,Aruba,ABW,1993,17.859
4,Aruba,ABW,1994,17.270
...,...,...,...,...
7387,Zimbabwe,ZWE,2013,35.768
7388,Zimbabwe,ZWE,2014,34.978
7389,Zimbabwe,ZWE,2015,33.981
7390,Zimbabwe,ZWE,2016,32.864


In [5]:
#we clean up the food production dataframe
food_production_raw = food_production_raw[['Country Name','Country Code', '1990','1991','1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']]
food_production = pd.DataFrame()
years = ['1990','1991','1992','1993','1994','1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']
for index, row in food_production_raw.iterrows():
    for year in years:
        food_production = food_production.append([[row[0],row[1],year,row[year]]])
food_production = food_production.rename(columns={0:'Country', 1:'Country Code', 2:'Year', 3:'Food Production'})
food_production = food_production.dropna()
food_production

,Country,Country Code,Year,Food Production
0,Afghanistan,AFG,1990,68.12
0,Afghanistan,AFG,1991,74.24
0,Afghanistan,AFG,1992,71.75
0,Afghanistan,AFG,1993,76.25
0,Afghanistan,AFG,1994,81.33
...,...,...,...,...
0,Zimbabwe,ZWE,2012,94.08
0,Zimbabwe,ZWE,2013,92.05
0,Zimbabwe,ZWE,2014,91.23
0,Zimbabwe,ZWE,2015,83.42


In [6]:
country_codes = currencies['Country_Code']
code_dict = {currency: code for (currency, code) in zip(currencies['Currency'], currencies['Country_Code'])}
birth_rates = birth_rates[birth_rates['Country Code'].isin(country_codes)]
food_production = food_production[food_production['Country Code'].isin(country_codes)]

#we prepare to cycle through our data sets and keep only overlapping values
currency_keys=[]
birth_keys=[]
food_keys=[]
#for index, row in currency_clean.iterrows():
#    row['Country Code'] = code_dict[row['Currency']]
#    currency_keys.append([row['Country Code'], row['Date']])
for index, row in birth_rates.iterrows():
    birth_keys.append([row['Country Code'], row['Year']])
for index, row in food_production.iterrows():
    food_keys.append([row['Country Code'], row['Year']])

In [7]:
#now we cycle through and remove countries that do not have overlapping values
for index, row in currency_clean.iterrows():
    row['Country Code'] = code_dict[row['Currency']]
    if [row['Country Code'], row['Date']] not in birth_keys:
        currency_clean.drop(index, inplace = True)
    elif [row['Country Code'], row['Date']] not in food_keys:
        currency_clean.drop(index, inplace = True)
for index, row in currency_clean.iterrows():
    row['Country Code'] = code_dict[row['Currency']]
    currency_keys.append([row['Country Code'], row['Date']])        

In [8]:
for index, row in birth_rates.iterrows():
    if [row['Country Code'], row['Year']] not in currency_keys:
        birth_rates.drop(index, inplace = True)
for index, row in food_production.iterrows():
    if [row['Country Code'], row['Year']] not in currency_keys:
        birth_rates.drop(index, inplace = True)

C:\Users\maste\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
#clean up and export
currency_clean = currency_clean.reset_index(drop = True)
birth_rates = birth_rates.reset_index(drop = True)
food_production = food_production.reset_index(drop = True)

In [10]:
currency_clean.to_csv('currency_clean')
birth_rates.to_csv('birth_rates_clean')
food_production.to_csv('food_production_clean')